In [1]:
import os
import numpy as np
import imageio
from matplotlib import pyplot as plt
from utils import *

In [2]:
tMax = 60
stockMin = 317
stockMax = 363
strike = 340

###
n = int((stockMax - stockMin)/0.01 + 1)
stock = np.linspace(stockMin, stockMax, n)

### t days until expiration
valueT = np.zeros((tMax+1, n))
deltaT = np.zeros((tMax+1, n))
deltaFactorT = np.zeros((tMax+1, n))
for t in range(tMax+1):
    value = BlackScholes(strike, stock, t)
    valueT[t] = value
    delta = BlackScholes(strike, stock+1, t) - value
    deltaT[t] = delta
    deltaFactorT[t] = stock[t]*delta/value

### Normalizing Data

# normalizing stock value
x = (stock - strike) / strike

/Users/peter/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:19: RuntimeWarning: divide by zero encountered in true_divide
/Users/peter/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:19: RuntimeWarning: invalid value encountered in true_divide


In [3]:
# Generating images
files = []
for t in range(tMax+1)[::-1]:
    fig, ax1 = plt.subplots(figsize = [9, 8], tight_layout = True)
    xlabel = r"$\bf{Value\ of\ the\ Underyling\ Asset}$"
    xlabel += "\n"
    xlabel += r"$\rm{(Percent\ difference\ between\ stock\ price\ and\ strike\ price)}$"
    plt.xlabel(xlabel, fontsize = 16)    

    title = r"$\bf{Delta\ Value\ of\ Call\ Option\ Contracts}$"
    title += "\n"
    title += r"$\rm{Days\ to\ Expiration:}$"
    title += " %2s" % t
    plt.title(title, family = "monospace", fontsize = 16)
    
    plt.xlim(x[0], x[-1])
    plt.xticks(fontsize = 14)
    plt.grid()
    plt.gca().set_xticklabels(['{:.0f}%'.format(x*100) for x in plt.gca().get_xticks()]);

    ax1.tick_params(axis='y', labelsize = 14, labelcolor='r')
    ax1.set_yticks(np.linspace(0, 1, 6))
    ax1.plot(x, deltaT[t], color = 'r', linewidth = 2.5)
    ylabel = r"$\bf{Delta}$"
    ylabel += "\n"
    ylabel += r"$\rm{(Change\ in\ option\ value\ due\ to\ \$1\ stock\ increase)}$"
    ax1.set_ylabel(ylabel, color = 'r', fontsize = 16)
    ax1.set_ylim(0, 1)
    
    ax2 = ax1.twinx()
    ax2.plot(x, deltaFactorT[t], color = 'b', linewidth = 2.5)
    ax2.tick_params(axis='y', labelsize = 14, labelcolor='b')
    ax2.set_yticks(np.linspace(0, 100, 6))
    ylabel = r"$\bf{Buying\ Power\ Factor}$"
    ylabel += "\n"
    ylabel += r"$\rm{(Delta\times\frac{stock\ price}{contract\ price})}$"
    ax2.set_ylabel(ylabel, color = 'b', fontsize = 16)
    ax2.set_ylim(0, 100)
    
    fname = "delta_%02d.png" % t
    files.append(fname)
    plt.savefig(fname, bbox_inches = "tight")
    plt.close()

In [4]:
### Making movie animation
images = []
for i in range(tMax+1):
    fname = files[i]
    # make first and last frames longer
    if i == 0 or i == tMax:
        for _ in range(9):
            images.append(imageio.imread(fname))
    #
    images.append(imageio.imread(fname))
imageio.mimsave('Delta.gif', images, duration=0.1)

### cleanup
for fname in files:
    os.remove(fname)